🔢 1. Training Terdistribusi dengan MirroredStrategy

In [46]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Load dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

# Distribusi di seluruh GPU
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = keras.Sequential([
        keras.layers.Conv2D(32, 3, activation="relu", input_shape=(28, 28, 1)),
        keras.layers.MaxPooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🧠 2. Logging & Checkpointing (untuk TensorBoard)

In [48]:
import os
log_dir = "./logs"
checkpoint_path = "checkpoints/mnist_model.weights.h5"

callbacks = [
    keras.callbacks.TensorBoard(log_dir=log_dir),
    keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, save_weights_only=True)
]

🚀 3. Train Model dengan Callbacks

In [49]:
model.fit(X_train, y_train, epochs=5, validation_split=0.1, callbacks=callbacks)

Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9017 - loss: 0.3210 - val_accuracy: 0.9800 - val_loss: 0.0673
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9830 - loss: 0.0586 - val_accuracy: 0.9830 - val_loss: 0.0587
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9889 - loss: 0.0364 - val_accuracy: 0.9845 - val_loss: 0.0528
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9934 - loss: 0.0232 - val_accuracy: 0.9850 - val_loss: 0.0508
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9959 - loss: 0.0140 - val_accuracy: 0.9860 - val_loss: 0.0563


💾 4. Simpan Model dalam Format SavedModel

In [52]:
model.save("saved_model/mnist_model.h5")

📡 5. Simulasi API TensorFlow Serving (lokal)

In [53]:
import json

# Load kembali model
loaded_model = keras.models.load_model("saved_model/mnist_model.h5")

# Uji dengan satu input
sample = X_test[0:1]
prediction = loaded_model.predict(sample)
predicted_class = np.argmax(prediction[0])

print("Predicted class:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step
Predicted class: 7
